In [16]:
import os
from langchain_groq import ChatGroq

# Define the LLM
os.environ["GROQ_API_KEY"] = "ENTER API KEY"

llm = ChatGroq(
    model_name="llama-3.1-8b-instant",
    temperature=0.7,
    max_tokens=200
)

In [17]:
# Simply call invoke method on the llm and pass in the input question.

#:#Step 2 - Invoke llm with a fixed text input

response = llm.invoke("Tell me one fact about space", temperature=0.7)
print("Scenario 1 Response - >")
print(response.pretty_print())

Scenario 1 Response - >
================================== Ai Message ==================================

One fascinating fact about space is that there is a giant storm on Jupiter called the Great Red Spot, which has been continuously raging for at least 187 years and possibly much longer.
None


In [18]:
# Define a prompt template to structure user inputs. Template allows us to collect inputs at runtime.

#:#Step 3 - Use String Prompt to accept text input. Here we create a template and
#declare a input variable {user_input} and {city}

from langchain_core.prompts import PromptTemplate

template = """You are a chatbot having a conversation with a human.
Human: {user_input} {city}
"""

In [19]:
#:#Step 4 - Here we create a Prompt using the template

prompt = PromptTemplate(input_variables=["user_input", "city"], template=template)

In [20]:
#:#Step 5 - Here we get a prompt value and print it.

prompt_val = prompt.invoke({"user_input":"Tell us in an Pirate tone about", "city":"Las Vegas"})
print("Prompt String is ->")
print(prompt_val.to_string())

Prompt String is ->
You are a chatbot having a conversation with a human.
Human: Tell us in an Pirate tone about Las Vegas



In [21]:
# Combine the prompt template and the OCI Generative AI model using the pipe operator. In this setup, the invoke method processes the input through the defined chain, producing the desired output.

#:#Step 6 - here we declare a chain that begins with a prompt, next llm

chain = prompt | llm

In [22]:
#:#Step 7 - Next we invoke a chain, get response and print it.

response = chain.invoke({"user_input":"Tell us in a priate tone about", "city":"New York"})

print("Scenario 2 Response - >")
print(response.pretty_print())

Scenario 2 Response - >
================================== Ai Message ==================================

(in a charming, pirate-like tone) Ahoy, matey! Ye be wantin' to know about the grand city o' New York, eh? Well, settle yerself down with a pint o' grog and listen close, for I be tellin' ye tales o' the Big Apple.

New York, the city that never sleeps, be a place o' wonder and magic. 'Tis a melting pot o' cultures, where the streets be filled with the sounds o' laughter, music, and the clinkin' o' glasses. From the bright lights o' Times Square to the peaceful greenery o' Central Park, this city be a treasure trove o' excitement and adventure.

Ye can stroll along the mighty Hudson River, takin' in the sights o' the Statue o' Liberty and the Brooklyn Bridge. Or, ye can explore the many museums and art galleries, where ye can see the works o' the masters and learn about the
None


In [23]:
# Define the chat prompt template.

#:#Step 8 - Use Chat Message Prompt to accept text input. Here we create a chat template and
#use HumanMessage and SystemMessage

from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a chatbot that explains in steps."),
        ("human", "{input}"),
    ]
)

In [24]:
#:#Step 9 - Here we get a prompt value and print it.

prompt_value = chat_prompt.invoke({"input":"What is microbiology?"})
print(prompt_value)
# messages=[SystemMessage(content='You are a chatbot that explains in steps.'), HumanMessage(content='What is microbiology?')]

messages=[SystemMessage(content='You are a chatbot that explains in steps.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is microbiology?', additional_kwargs={}, response_metadata={})]


In [25]:
# Combine the chat prompt template and the OCI Generative AI model using the pipe operator. In this setup, the invoke method processes the input through the defined chain, producing the desired output.

#:#Step 10 - create another chain, get a response and print it.

chain1 = chat_prompt | llm
response = chain1.invoke({"input": "What's the New York culture like?"})

print("Scenario 3 Response - >")
print(response.pretty_print())

Scenario 3 Response - >
================================== Ai Message ==================================

Exploring New York culture can be a fascinating topic. Here's a step-by-step breakdown:

**Step 1: Diversity and Melting Pot**
New York City is a melting pot of cultures, with over 8.4 million people from different ethnic backgrounds, speaking over 800 languages. This diversity is reflected in the food, music, art, and lifestyle of the city.

**Step 2: Fast-paced and Dynamic**
New York is known for its fast-paced and dynamic environment. People are often in a hurry, and the city never sleeps. This energy is reflected in the 24/7 lifestyle, with restaurants, shops, and entertainment options available around the clock.

**Step 3: Food Culture**
New York's food culture is a reflection of its diversity. You can find authentic Chinese, Italian, Mexican, Indian, and many other cuisines in the city. Some popular New York dishes include pizza, bagels, pastrami sandwiches, and black and whi

In [26]:
from langgraph.graph import MessageGraph
from langgraph.checkpoint.memory import InMemorySaver

# 1. Define the checkpointer (memory)
# This is the same as your code.
checkpointer = InMemorySaver()

# 2. Define the graph
# MessageGraph is the simplest way to build a stateful chatbot
graph_builder = MessageGraph()

# 3. Define the function that calls your LLM
# This function will be a "node" in the graph
def llm_node(state):
    # 'state' is the list of messages from memory
    # We pass this entire history to the llm
    return llm.invoke(state)

# 4. Add the node to the graph
graph_builder.add_node("llm", llm_node)

# 5. Define the graph's start and end points
graph_builder.set_entry_point("llm")
graph_builder.set_finish_point("llm")

# 6. Compile the graph into the final "agent"
# THIS is the step that connects the agent to the memory
agent = graph_builder.compile(checkpointer=checkpointer)

print("Chat agent with memory is compiled and ready.")

Chat agent with memory is compiled and ready.


/tmp/ipython-input-916435198.py:10: LangGraphDeprecatedSinceV10: MessageGraph is deprecated in LangGraph v1.0.0, to be removed in v2.0.0. Please use StateGraph with a `messages` key instead. Deprecated in LangGraph V1.0 to be removed in V2.0.
  graph_builder = MessageGraph()


In [27]:
#:#Step 12 - Here we ask our first question and print response and memory contents

# Define the config for the conversation thread
conversation_id = "my-first-graph-chat"
config = {"configurable": {"thread_id": conversation_id}}

# The input is a list of messages.
msg1 = [("user", "Hi! My name is Ayush.")]

print("--- Turn 1 ---")
# 1. Invoke the agent
response_messages_1 = agent.invoke(msg1, config=config)

# Get the last message (the AI's reply)
ai_response_1 = response_messages_1[-1]
print(f"AI Response: {ai_response_1.content}")

# 2. Get the full history from the checkpointer's memory
memory_state_1 = checkpointer.get(config)

# --- THIS IS THE FIX ---
# The 'memory_state_1' variable IS the list of messages
print(f"\nMemory Contents: {memory_state_1['channel_values']['__root__']}")

--- Turn 1 ---
AI Response: Hello Ayush, it's nice to meet you. Is there something I can help you with or would you like to chat?

Memory Contents: [HumanMessage(content='Hi! My name is Ayush.', additional_kwargs={}, response_metadata={}, id='5e89a686-537e-4b3a-823b-42e242c271b3'), AIMessage(content="Hello Ayush, it's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 43, 'total_tokens': 70, 'completion_time': 0.024533283, 'prompt_time': 0.002030763, 'queue_time': 0.022542787, 'total_time': 0.026564046}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_6b5c123dd9', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--4805ec3c-deb9-400e-873c-060a89621d6a-0', usage_metadata={'input_tokens': 43, 'output_tokens': 27, 'total_tokens': 70})]


In [28]:
#:#Step 13 - Here we ask our followup question and print response and memory contents

# The config is the same. We just send the new message.
msg2 = [("user", "Can you tell what is my name?")]

print("\n--- Turn 2 ---")
# 1. Invoke the agent again
response_messages_2 = agent.invoke(msg2, config=config) # Use the same config

# Get the last message (the AI's reply)
ai_response_2 = response_messages_2[-1]
print(f"AI Response: {ai_response_2.content}")

# 2. Get the updated history from the checkpointer's memory
memory_state_2 = checkpointer.get(config)

# --- THIS IS THE FIX ---
# The 'memory_state_2' variable IS the list of messages
print(f"\nMemory Contents: {memory_state_2}")


--- Turn 2 ---
AI Response: Your name is Ayush.

Memory Contents: {'v': 4, 'ts': '2025-11-04T09:19:33.089098+00:00', 'id': '1f0b95f5-fed1-6ab1-8004-df7d62b8ac6a', 'channel_versions': {'__start__': '00000000000000000000000000000005.0.4501086194323214', '__root__': '00000000000000000000000000000006.0.8149478015795615', 'branch:to:llm': '00000000000000000000000000000006.0.8149478015795615'}, 'versions_seen': {'__input__': {}, '__start__': {'__start__': '00000000000000000000000000000004.0.5859061446560523'}, 'llm': {'branch:to:llm': '00000000000000000000000000000005.0.4501086194323214'}}, 'updated_channels': ['__root__'], 'channel_values': {'__root__': [HumanMessage(content='Hi! My name is Ayush.', additional_kwargs={}, response_metadata={}, id='5e89a686-537e-4b3a-823b-42e242c271b3'), AIMessage(content="Hello Ayush, it's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'p